# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# out put file and create DF and Drop unamed columns
output_data_file = "weather_df.csv"
weather_df = pd.read_csv(output_data_file, index_col=0)
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City ID,,,,,,,,,
0,Ushuaia,-54.8000,-68.3000,44.60,70,40,34.52,AR,1618590338
1,Nioro,13.3500,-15.7500,104.59,10,50,6.73,GM,1618590608
2,Albany,42.6001,-73.9662,42.01,76,100,1.01,US,1618590331
3,Kirovskiy,45.0917,133.5136,42.64,88,93,5.91,RU,1618590609
4,Leeuwarden,53.2014,5.8086,50.00,61,7,13.80,NL,1618590609
...,...,...,...,...,...,...,...,...,...
549,Coyhaique,-45.5752,-72.0662,59.00,47,40,10.36,CL,1618590766
550,Mbandaka,0.0487,18.2603,76.78,84,100,4.97,CD,1618590766
551,Kupang,-10.1667,123.5833,76.21,88,83,6.93,ID,1618590766


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
# use Lat and lng and Humidty as weight
loc = weather_df[["Lat","Lng"]]
# couldnt get to work without float
hum = weather_df["Humidity"].astype(float)
# added max after re reading instructions
max_hum = hum.max()


In [13]:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(loc,weights = hum,dissipating=False,max_intensity = max_hum,point_radius=5)
fig.add_layer(heatmap_layer)
plt.savefig("heatmaplayer_fig_1.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# A max temperature lower than 80 degrees but higher than 70. - check
# Wind speed less than 10 mph. -- done
# Zero cloudiness. -  done
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit 
# the number of rows returned by your API requests to a reasonable number.
# what is reasonable?

# did not work the way i want it to

# weather_condition_df = pd.DataFrame(weather_df, columns=["City","Max Temp","Wind Speed","Cloudiness"])
# max_temp = (weather_condition_df["Max Temp"]< 80)&(weather_condition_df["Max Temp"]>=70)
# wind_spd=weather_condition_df["Wind Speed"] < 10
# clouds=weather_condition_df["Cloudiness"] == 0
# #create with 3 conditions - had to add & commas cause error
# weather_condition_df[max_temp&wind_spd&clouds]

weather_condition_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & (weather_df["Cloudiness"] == 0)& (weather_df["Wind Speed"] <10), :]
weather_condition_df = weather_condition_df.dropna(how='any')
weather_condition_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City ID,,,,,,,,,
79,Saint-Philippe,-21.3585,55.7679,78.80,78,0,3.44,RE,1618590630
107,Saint-Joseph,-21.3667,55.6167,78.80,78,0,3.44,RE,1618590637
161,Pueblo Nuevo,23.3833,-105.3833,74.88,10,0,7.36,MX,1618590653
295,Vallenar,-28.5708,-70.7581,78.42,41,0,6.80,CL,1618590691
419,Saint-Pierre,-21.3393,55.4781,78.80,78,0,3.44,RE,1618590728


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = pd.DataFrame(weather_condition_df, columns =["City","Lat","Lng","Country"])
hotel_df["Hotel Name"] = " "
hotel_df.head()


,City,Lat,Lng,Country,Hotel Name
City ID,,,,,
79,Saint-Philippe,-21.3585,55.7679,RE,
107,Saint-Joseph,-21.3667,55.6167,RE,
161,Pueblo Nuevo,23.3833,-105.3833,MX,
295,Vallenar,-28.5708,-70.7581,CL,
419,Saint-Pierre,-21.3393,55.4781,RE,


In [16]:
params = {
    "radius": 5000,
    "types" : "hotel",
    "key": g_key
    }

for index,row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    params["location"]= f"{lat},{lng}"
 
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params=params)
    json = requested.json()
    try:
        hotel_df.loc[index,"Hotel Name"] = json["results"][0]["name"]
        #hotellist.append(requested.json()['results'][0]['name'])
    except (KeyError, IndexError):
        print('Missing field')

hotel_df.head()

,City,Lat,Lng,Country,Hotel Name
City ID,,,,,
79,Saint-Philippe,-21.3585,55.7679,RE,Saint-Philippe
107,Saint-Joseph,-21.3667,55.6167,RE,Saint-Joseph
161,Pueblo Nuevo,23.3833,-105.3833,MX,Ваље де ла Буфа
295,Vallenar,-28.5708,-70.7581,CL,Vallenar
419,Saint-Pierre,-21.3393,55.4781,RE,Saint Pierre


In [17]:
#NOTE: Do not change any of the code in this cell

#Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
#Store the DataFrame Row
#NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
plt.savefig("heatmapfig_2.png")
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>